In [ ]:
# Install required libraries

import requests
import json
import time
import datetime
from urllib.request import urlopen
import requests
import warnings
from copy import copy
import sys

import numpy as np
import pandas as pd
import seaborn as sns
import osmnx as ox
import networkx as nx
import geopandas as gpd

from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from pyproj import CRS
import os
import movingpandas as mpd
from shapely.geometry import Point

# Suppress deprecation warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely import wkt

# Function to convert the datatypes into original
def convert_dtypes(df):
    # Convert 'timestamp' to datetime64[ns]
    df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
    
    # Convert 'geometry' to geometry type if using GeoPandas
    df = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")
    
    return df
    
# Read and process the CSV file into a DataFrame
geodata_lytton_full_sumo_timesorted = pd.read_csv('C:/Work/Wildfire/data_lytton_br/geodata_lytton_full_sumo_timesorted.csv')
geodata_lytton_full_sumo_timesorted['geometry'] = geodata_lytton_full_sumo_timesorted['geometry'].apply(wkt.loads)
geodata_lytton_full_timesorted = geodata_lytton_full_sumo_timesorted.drop(columns=['sumo_x', 'sumo_y', 'edge_id'])
geodata_lytton_full_timesorted = gpd.GeoDataFrame(geodata_lytton_full_timesorted, geometry='geometry', crs="EPSG:4326")

# Create a duplicate of the 'timestamp' column
geodata_lytton_full_timesorted['t'] = geodata_lytton_full_timesorted['timestamp']
geodata_lytton_full_timesorted.set_index('t', inplace=True)

# Convert to proper datatype
geodata_lytton_full_timesorted = convert_dtypes(geodata_lytton_full_timesorted)

# Display the DataFrame
display(geodata_lytton_full_timesorted)

In [ ]:
geodata_lytton_full_2 = geodata_lytton_full_timesorted[(geodata_lytton_full_timesorted['timestamp']<'2021-07-03') & (geodata_lytton_full_timesorted['timestamp']>'2021-07-02')]
geodata_lytton_full_2

In [ ]:
import osmnx as ox
import geopandas as gpd
from shapely.geometry import box

# Load the shapefile
shapefile = gpd.read_file("C:/Work/Wildfire/data_lytton_br/gis/Lytton_Square.shp")

# Reproject the shapefile to WGS 84 (EPSG:4326) for SUMO
shapefile = shapefile.to_crs(epsg=4326)

# Extract the bounding box from the shapefile
bounding_box = shapefile.total_bounds
print(bounding_box)

# Create a bounding box object (optional)
bbox_geom = box(*bounding_box)

# Assuming shapefile has one polygon
polygon = shapefile.geometry.iloc[0] 

# Turn off simplify to obtain a detailed graph; this is important for proper map matching
roads_lytton_square = ox.graph_from_polygon(polygon, network_type='drive', simplify=False)

# Plot the graph
ox.plot_graph(roads_lytton_square, show=True)

In [ ]:
# Import roadmap from OSM into UXSim and visualize

from uxsim import *
from uxsim.OSMImporter import OSMImporter

# Convert the OSMNX data for UXSim (Source: https://github.com/toruseo/UXsim/blob/main/uxsim/OSMImporter/OSMImporter.py)
def prepare_for_uxsim(G):
    default_number_of_lanes_mortorway=3
    default_number_of_lanes_trunk=3
    default_number_of_lanes_primary=2
    default_number_of_lanes_secondary=2
    default_number_of_lanes_residential=1
    default_number_of_lanes_tertiary=1, 
    default_number_of_lanes_others=1
    default_maxspeed_mortorway=100
    default_maxspeed_trunk=60
    default_maxspeed_primary=50
    default_maxspeed_secondary=50
    default_maxspeed_residential=30
    default_maxspeed_tertiary=30, 
    default_maxspeed_others=30

    node_dict = {}
    for n in G.nodes:
        nd = G.nodes[n]
        node_dict[n]=[n, nd["x"], nd["y"]]

    links = []
    nodes = {}
    for e in G.edges:
        ed = G.get_edge_data(e[0], e[1])[0]
            
        if "highway" in ed:
            road_type = ed["highway"]
            try:
                name = ed["name"]
                if type(name) == list:
                    name = name[0]
                osmid = ed["osmid"]
                if type(osmid) == list:
                    osmid = osmid[0]
                name += "-"+str(osmid)
            except:
                name = ""
                osmid = ""
            try:
                lanes = int(ed["lanes"])
            except:
                try:
                    if "mortorway" in road_type:
                        lanes = default_number_of_lanes_mortorway
                    elif "trunk" in road_type:
                        lanes = default_number_of_lanes_trunk
                    elif "primary" in road_type:
                        lanes = default_number_of_lanes_primary
                    elif "secondary" in road_type:
                        lanes = default_number_of_lanes_secondary
                    elif "residential" in road_type:
                        lanes = default_number_of_lanes_residential
                    elif "tertiary" in road_type:
                        lanes = default_number_of_lanes_tertiary
                    else:
                        lanes = default_number_of_lanes_others
                except:
                    lanes = default_number_of_lanes_others
                if lanes < 1:
                    lanes = 1
            try:
                maxspeed = float(ed["maxspeed"])/3.6
            except:
                try:
                    if "mortorway" in road_type:
                        maxspeed = default_maxspeed_mortorway/3.6
                    elif "trunk" in road_type:
                        maxspeed = default_maxspeed_trunk/3.6
                    elif "primary" in road_type:
                        maxspeed = default_maxspeed_primary/3.6
                    elif "secondary" in road_type:
                        maxspeed = default_maxspeed_secondary/3.6
                    elif "residential" in road_type:
                        maxspeed = default_maxspeed_residential/3.6
                    elif "tertiary" in road_type:
                        maxspeed = default_maxspeed_tertiary/3.6
                    else:
                        maxspeed = default_maxspeed_others/3.6
                except:
                    maxspeed = default_maxspeed_others/3.6
            

            links.append([name, e[0], e[1], lanes, maxspeed]) # name, from, to, number_of_lanes, maxspeed
            #links.append([name, e[0], e[1], 1, maxspeed]) # name, from, to, number_of_lanes, maxspeed
            nodes[e[0]] = node_dict[e[0]]
            nodes[e[1]] = node_dict[e[1]]

    nodes = list(nodes.values()) 

    return nodes, links

nodes, links = prepare_for_uxsim(roads_lytton_square)

# Simplifying the node
K = World(
    name="",
    deltan=5,
    tmax=7200,
    print_mode=1, save_mode=1, show_mode=0, 
    random_seed=0
)

nodes2, links2 = OSMImporter.osm_network_postprocessing(nodes, links, node_merge_threshold=0.0005, node_merge_iteration=5, enforce_bidirectional=True)
OSMImporter.osm_network_to_World(K, nodes2, links2, default_jam_density=0.2, coef_degree_to_meter=111000)

In [ ]:
# Visualize the simplified node
OSMImporter.osm_network_visualize(nodes2, links2, show_link_name=0)

In [ ]:
# Visualize the simplified node - zoomed in
OSMImporter.osm_network_visualize(nodes2, links2, show_link_name=0, xlim=[-121.6, -121.5], ylim=[50.2, 50.28], figsize=(6,6))

In [ ]:
import pandas as pd

# Assuming df is your DataFrame, and it’s already read into memory
geodata_lytton_full_timesorted = geodata_lytton_full_timesorted.sort_values(by=['device_id', 'timestamp'])

# Initialize the list to store output data
geodata_lytton_full_timesorted_list = []

# Loop through each device_id group
for device_id, group in geodata_lytton_full_timesorted.groupby('device_id'):
    previous_row = None  # Variable to keep track of the previous row
    for idx, row in group.iterrows():
        # If previous row exists and `geometry_matched` changes
        if previous_row is not None and row['geometry_matched'] != previous_row['geometry_matched']:
            # Prepare the row in the specified format
            output_row = [
                previous_row['device_id'],       # device
                previous_row['timestamp'],       # time
                previous_row['long_matched'],    # x_orig
                previous_row['lat_matched'],     # y_orig
                0,                               # radious_orig
                row['long_matched'],             # x_dest
                row['lat_matched'],              # y_dest
                0.5,                             # radious_dest
                previous_row['depart'],          # t_start
                row['depart'],                   # t_end
                -1,                              # static -1
                500,                             # static 500
                None,                            # static None
                True                             # static True
            ]
            # Append the row to the output list
            geodata_lytton_full_timesorted_list.append(output_row)
        
        # Update the previous row to the current row for the next iteration
        previous_row = row

# output now contains the required 2D list of entries
geodata_lytton_full_timesorted_list[:5]

In [ ]:
def sort_by_time_value(two_d_list):
    # Sort the list based on the time value of each sublist
    sorted_list = sorted(two_d_list, key=lambda x: x[1])
    return sorted_list

geodata_lytton_full_timesorted_list = sort_by_time_value(geodata_lytton_full_timesorted_list)
geodata_lytton_full_timesorted_list[:5]

In [ ]:
# Mapping depart time with actual GPS time
import pandas as pd
from datetime import timedelta

# Define your start_time (you would replace this with your actual start time)
start_time = pd.to_datetime("2021-06-18 00:00:00")

# Set the intervals for depart values (0, 50, 100, 150, etc.)
depart_interval = 40  # You can add more intervals as needed
depart_start = 0
depart_end = 400

# Calculate the corresponding timestamps for each depart interval
timestamp_results = []

for depart in list(range(depart_start, depart_end, depart_interval)):
    # Calculate the time delta in seconds (depart in hours * 3600 seconds)
    time_delta = timedelta(hours=depart)
    
    # Calculate the corresponding timestamp by adding the time delta to start_time
    timestamp = start_time + time_delta
    print('Depart:',depart,'Timestamp',timestamp)

In [ ]:
# Adding the demands to the world K

# Assuming K is your traffic simulation object and adddemand_area2area is a method of K
volume = 500

# Iterate over the output list generated in the previous step
for entry in geodata_lytton_full_timesorted_list:
    # Unpack parameters from the entry
    device, time, x_orig, y_orig, radius_orig, x_dest, y_dest, radius_dest, t_start, t_end, *_ = entry
    
    # Call the adddemand_area2area method with unpacked parameters and set volume to 500
    K.adddemand_area2area2(
        x_orig, y_orig, radius_orig,
        x_dest, y_dest, radius_dest,
        t_start, t_end, volume
    )

In [ ]:
K.exec_simulation()

In [ ]:
K.analyzer.print_simple_stats()
K.analyzer.network_anim(animation_speed_inverse=15, detailed=0, network_font_size=0)
K.analyzer.network_fancy(animation_speed_inverse=15, sample_ratio=0.1, interval=10, trace_length=5)

from IPython.display import display, Image
with open("out/anim_network0.gif", "rb") as f:
    display(Image(data=f.read(), format='png'))
with open("out/anim_network_fancy.gif", "rb") as f:
    display(Image(data=f.read(), format='png'))
